In [ ]:
import urllib
from bs4 import BeautifulSoup
url = raw_input("url: \n")
html = urllib.urlopen(url).read()
soap = BeautifulSoup(html,"lxml")
tags = soap('span')
count = 0
sum = 0
for tag in tags:
    count = count + 1
#    print 'content:', int(tag.contents[0])
    sum =sum + int(tag.contents[0])
print 'count',count
print 'sum',sum


In [ ]:
print 'hello'

In [ ]:
import urllib
from bs4 import BeautifulSoup
urllink = raw_input('Enter URL:')
count = int(raw_input('Enter count:'))
position = int(raw_input('Enter position:'))

for i in range(count):
    html = urllib.urlopen(urllink).read()
    soup = BeautifulSoup(html, "lxml")
    print soup, type(soup)
    tags = soup('a')
    print i
#    print tags
#    for j in range(position) :
#        if j == (position-1):
    urllink = tags[position-1].get('href',None)
    print urllink



In [ ]:
import urllib
import xml.etree.ElementTree as ET
xmlurl = raw_input("enter location:")
print 'Retrieving', xmlurl
xmlstr = urllib.urlopen(xmlurl).read()
print 'Retrieved',len(xmlstr),'characters'
xmltree = ET.fromstring(xmlstr)
comments = xmltree.findall('comments/comment')
count = 0
sum = 0
for comment in comments:
    count = count + 1
    sum = sum + int(comment.find('count').text)

print 'count:', count
print 'sum:', sum

In [ ]:
import urllib
import json
url = raw_input("Enter location:")
print "Retrieving",url
urldata = urllib.urlopen(url).read()
print "Retrieved",len(urldata),"characters"
count =0
sum =0
try: js = json.loads(str(urldata))
except: 
    print '======= failure to retrieve ======='
    print urldata
    quit()
#print json.dumps(js,indent = 4)

for comment in js['comments']:
    count = count + 1
    sum = sum + int(comment['count'])

print count
print sum
    


In [ ]:
import urllib
import json
import re
location = raw_input("enter location:")
urllocsearch='http://python-data.dr-chuck.net/geojson?'+urllib.urlencode({'sensor':'false','address':location})
#print urllocsearch
print 'Retrieving', urllocsearch
searchresult=urllib.urlopen(urllocsearch).read()
print "retrieved",len(searchresult),'characters'
js=json.loads(str(searchresult))
#print json.dumps(js,indent =4)
print js['results'][0]['place_id']

In [ ]:
import sqlite3
import urllib
import re

conn = sqlite3.connect('python4edemo.sqlite')
cur = conn.cursor()

cur.execute('''
DROP TABLE IF EXISTS Counts''')

cur.execute('''
CREATE TABLE Counts (org TEXT, count INTEGER)''')

urllink = raw_input('Enter Url: ')
fh = urllib.urlopen(urllink)
for line in fh:
    org = re.findall('^From: .+@(\S+)',line)
    if len(org) == 0:continue
#From stephen.marquard@uct.ac.za Sat Jan  5 09:14:16 2008    
#    print email
#    print org
    cur.execute('SELECT count FROM Counts WHERE org = ? ', (org[0], ))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count) 
                VALUES ( ?, 1 )''', ( org[0], ) )
    else : 
        cur.execute('UPDATE Counts SET count=count+1 WHERE org = ?', 
            (org[0], ))
    # This statement commits outstanding changes to disk each 
    # time through the loop - the program can be made faster 
    # by moving the commit so it runs only after the loop completes
conn.commit()
# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC'

print'===================='
print "Counts:"
for row in cur.execute(sqlstr) :
    print str(row[0]), row[1]

cur.close()

In [ ]:
import sqlite3
import xml.etree.ElementTree as ET

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
#        print child.tag, child.text
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
DROP TABLE IF EXISTS Genre;
''')

cur.executescript('''
CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')

fname = raw_input('enter file name')
if len(fname) < 2: fname ='D:/Python study/tracks/Library.xml'

xmlhandle = ET.parse(fname)
tracknodes = xmlhandle.findall('dict/dict/dict')
for track in tracknodes:
    if lookup(track,'Track ID') == None: continue
    title = lookup(track,'Name')
    album = lookup(track,'Album')
    artist = lookup(track,'Artist')
    genre = lookup(track,'Genre')
    lenth = lookup(track,'Total Time')
    rating = lookup(track,'Rating')
    count = lookup(track, 'Play Count')
    if title is None or artist is None or album is None or genre is None: 
        continue
    print title,',',album,',',artist,',',genre

#insert artist
    cur.execute('''insert or ignore into Artist (name) values (?)''',(artist,))
    cur.execute('''select id from Artist where name = ?''',(artist,))
    artist_id = cur.fetchone()[0]
#insert genre
    cur.execute('''insert or ignore into Genre (name) values (?)''',(genre,))
    cur.execute('''select id from Genre where name = ?''',(genre,))
    genre_id = cur.fetchone()[0]
#insert album
    cur.execute('''insert or ignore into Album (artist_id,title) values (?,?)''',(artist_id,album))
    cur.execute('''select id from Album where title = ?''',(album,))
    album_id = cur.fetchone()[0]
#insert track
    cur.execute('''insert or replace into Track (title,album_id,genre_id,len,rating,count) values (?,?,?,?,?,?)''',
                (title,album_id,genre_id,lenth,rating,count))
#    print trackid,name,artist
    
    conn.commit()
    


In [ ]:
import json
import sqlite3

conn = sqlite3.connect('rosterdb.sqlite')
cur = conn.cursor()

# Do some setup
cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;

CREATE TABLE User (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT UNIQUE
);

CREATE TABLE Course (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title  TEXT UNIQUE
);

CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id)
)
''')

fname = raw_input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'D:/Python study/roster_data.json'

# [
#   [ "Charley", "si110", 1 ],
#   [ "Mea", "si110", 0 ],

str_data = open(fname).read()
json_data = json.loads(str_data)

for entry in json_data:

    name = entry[0];
    title = entry[1];
    role = entry[2];
    print name, title, role

    cur.execute('''INSERT OR IGNORE INTO User (name) 
        VALUES ( ? )''', ( name, ) )
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Course (title) 
        VALUES ( ? )''', ( title, ) )
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id,role) VALUES ( ?, ?,? )''', 
        ( user_id, course_id ,role) )

    conn.commit()